In [1]:
###########################
# google drive 저장/로딩
###########################
#구글 드라이브 저장
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def doGoodleDriveAuth():
    # 1. Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    print('gauth:',gauth)
    gauth.credentials = GoogleCredentials.get_application_default()

    drive = GoogleDrive(gauth)
    print('drive:', drive)
    return drive

drive = doGoodleDriveAuth()

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html
def gcp_upload(file_path, w_file_name, drive=drive):
    # 특정 폴더 안으로 파일 삽입
    uploaded = drive.CreateFile({'title': w_file_name}) #, "parents": [{"kind": "drive#fileLink","id": 'jukyellow@gmail.com'}]})
    uploaded.SetContentString(w_file_name)
    uploaded.SetContentFile(file_path + w_file_name)
    uploaded.Upload()
    print('Uploaded file with ID {}'.format(uploaded.get('id')))

gauth: <pydrive.auth.GoogleAuth object at 0x7f4361e31da0>
drive: <pydrive.drive.GoogleDrive object at 0x7f436e769278>


In [0]:
#구글 드라이브에서 다운로드
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
import io
from io import BytesIO 
from googleapiclient.http import MediaIoBaseDownload

TEMP_PATH = '/tmp/'
def gcp_download(file_name, key):
    #3. 모델 다운로드
    #https://drive.google.com/open?id=1TlvbayGRCjAI6bOZrUYMmv6g6b95rnRM
    request = drive_service.files().get_media(fileId=key)

    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
    done = False
    while done is False:
      status, done = downloader.next_chunk()
      if status:
          print("Download %%%d%%." % int(status.progress() * 100))
      print("Download Complete!")
    downloaded.seek(0)

    with open(TEMP_PATH + file_name, 'wb') as f:
        f.write(downloaded.read())

In [3]:
#https://keras.io/examples/mnist_cnn/
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 5s 79us/step - loss: 0.2652 - accuracy: 0.9178 - val_loss: 0.0614 - val_accuracy: 0.9797
Epoch 2/12
60000/60000 [==============================] - 3s 54us/step - loss: 0.0891 - accuracy: 0.9732 - val_loss: 0.0431 - val_accuracy: 0.9850
Epoch 3/12
60000/60000 [==============================] - 3s 54us/step - loss: 0.0669 - accuracy: 0.9796 - val_loss: 0.0335 - val_accuracy: 0.9887
Epoch 4/12
60000/60000 [==============================] - 3s 53us/step - loss: 0.0547 - accuracy: 0.9831 - val_loss: 0.0347 - val_accuracy: 0.9877
Epoch 5/12
60000/60000 [==============================] - 3s 53us/step - loss: 0.0470 - accuracy: 0.9858 - val_loss: 0.0290 - val_accuracy: 0.9900
Epoch 6/12
60000/60000 [==============================] - 3s 53us/step - loss: 0.0426 - accuracy: 0.9873 - val_loss: 0.0276 - val_accuracy: 0.9900
Epoch 7/12
60000/60000 [==============================] - 3s 54us/st

In [5]:
#모델 저장/로딩 테스트
from keras.models import load_model
model_name = 'MODEL_SAVE_LOADING_TEST_MINIST.h5'
file_path = './'

model.save(model_name)
gcp_upload(file_path, model_name) #gcp root경로에 저장됨

Uploaded file with ID 1KT_qKfn6htBzw9DErJ6g4S4rrhwk9AE9


In [7]:
gcp_key = '1KT_qKfn6htBzw9DErJ6g4S4rrhwk9AE9'
gcp_download(model_name, gcp_key) #temp 경로에 다운로드
model = load_model(TEMP_PATH + 'MODEL_SAVE_LOADING_TEST_MINIST.h5') 

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Download %100%.
Download Complete!
Test loss: 0.02584999199801441
Test accuracy: 0.9914000034332275
